In [ ]:
#%pip install apify_client

In [2]:
# Dependancies
import pandas as pd
from apify_client import ApifyClient
import time
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [7]:
# Read post data
data = pd.read_excel("post.xlsx")
data.head(3)

,media/0/__isMedia,media/0/__typename,media/0/accent_color,media/0/animated_image_caption,media/0/audio_availability,media/0/audio_settings,media/0/audio_user_preferred_language,media/0/autoplay_gating_result,media/0/breakingStatus,media/0/broadcast_id,...,media/2/video_player_shaka_performance_logger_init2/per_session_sampling_rate,media/2/video_player_shaka_performance_logger_should_sample,media/2/video_status_type,media/2/viewer_autoplay_setting,media/2/width,url,text,likes,comments,shares
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://www.facebook.com/ibrahimamaigabf/posts...,"«Je ne perds jamais, soit je gagne, soit j'app...",1811,56,18
1,NaN,Video,NaN,NaN,AVAILABLE,NaN,en,all_page_organic_allowed,False,NaN,...,NaN,NaN,NaN,NaN,NaN,https://www.facebook.com/ibrahimamaigabf/video...,Triste.,2521,1111,530
2,Photo,Photo,FF000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://www.facebook.com/ibrahimamaigabf/posts...,NaN,2457,441,95


In [8]:
# Choosing important columns
data = data[["url", "text", "likes", "comments", "shares"]]
data.head(3)

,url,text,likes,comments,shares
0,https://www.facebook.com/ibrahimamaigabf/posts...,"«Je ne perds jamais, soit je gagne, soit j'app...",1811,56,18
1,https://www.facebook.com/ibrahimamaigabf/video...,Triste.,2521,1111,530
2,https://www.facebook.com/ibrahimamaigabf/posts...,NaN,2457,441,95


In [13]:
# Collect posts urls
urls = list(data["url"])

In [14]:
urls

['https://www.facebook.com/ibrahimamaigabf/posts/pfbid0LiK3A8uop129UP42AG9sa7trg3NHobGyn2ZVaPheH5QzZ9t4pyzyvX7dfu2rGj5fl',
 'https://www.facebook.com/ibrahimamaigabf/videos/1099971173944828/',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid0rTqBUHiv3vZDBuaVzYPha6R8brJPzYnhkY1eDcSHUUi8PuApfQrHgcgCeTiQfEPBl',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid02b7mW9KjYQYeFr5vzcbd5xwgV9ntrp6yTo5w1zRYBfpwKypYpEjojPcAHd9PjsFELl',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid02dYyc7Nc8nicHUadmFmf5fCaYZ1MtLd3K4YguWJ3xojp9Gr8FXVUquG56mVEwynUwl',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid0ckte351QT6NqQecc8KtHipcXmQ5dW6mZcKy4nVB6SNubnFupwpRafLkv861dExhel',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid0snAoVBvUph1CGj8NwbJnD1sqXezP8T5NBcsGLVwcYhU2w62pugFJAY2VJK4S5uUTl',
 'https://www.facebook.com/ibrahimamaigabf/videos/1230999364113693/',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid02t1VWih6pehgvNg9gcrTpYYkn4i4CNoy4TvUu5yEi72y7k85DT5Sbspe3oEfoqGWU

### Comments Scraping

In [ ]:
# Config
API_TOKEN = os.getenv("APIFY_API_TOKEN")
ACTOR_ID = os.getenv("COMMENT_ACTOR_ID")
OUTPUT_CSV = "facebook_comments.csv"  # Fichier de sortie
MAX_RESULTS = 100                 # Nombre max de commentaires par post
INCLUDE_NESTED = True           # Inclure les réponses aux commentaires

In [16]:
client = ApifyClient(API_TOKEN)

In [17]:
all_comments = []

for url in urls:
    print(f"📥 Récupération des commentaires pour : {url}")

    # Préparer l'entrée pour l'Actor
    run_input = {
        "startUrls": [{"url": url}],
        "resultsLimit": MAX_RESULTS,
        "includeNestedComments": INCLUDE_NESTED,
        "viewOption": "RANKED_UNFILTERED"
    }

    # Lancer l'Actor
    try:
        run = client.actor(ACTOR_ID).call(run_input=run_input)
    except Exception as e:
        print(f"⚠️ Erreur pour {url}: {e}")
        continue

    # Parcourir les résultats
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        comment_data = {
            "post_url": url,
            "comment_id": item.get("id"),
            "comment_text": item.get("text"),
            "comment_author": item.get("author", {}).get("name"),
            "comment_time": item.get("timestamp"),
            "comment_reactions": item.get("reactionsCount"),
            "reply_count": item.get("replyCount"),
        }
        all_comments.append(comment_data)

    # Pause légère pour éviter de saturer l'API
    time.sleep(3)

📥 Récupération des commentaires pour : https://www.facebook.com/ibrahimamaigabf/posts/pfbid0LiK3A8uop129UP42AG9sa7trg3NHobGyn2ZVaPheH5QzZ9t4pyzyvX7dfu2rGj5fl
⚠️ Erreur pour https://www.facebook.com/ibrahimamaigabf/posts/pfbid0LiK3A8uop129UP42AG9sa7trg3NHobGyn2ZVaPheH5QzZ9t4pyzyvX7dfu2rGj5fl: This API end-point can only be accessed using the following HTTP methods: OPTIONS,GET,PUT,DELETE
📥 Récupération des commentaires pour : https://www.facebook.com/ibrahimamaigabf/videos/1099971173944828/
⚠️ Erreur pour https://www.facebook.com/ibrahimamaigabf/videos/1099971173944828/: This API end-point can only be accessed using the following HTTP methods: OPTIONS,GET,PUT,DELETE
📥 Récupération des commentaires pour : https://www.facebook.com/ibrahimamaigabf/posts/pfbid0rTqBUHiv3vZDBuaVzYPha6R8brJPzYnhkY1eDcSHUUi8PuApfQrHgcgCeTiQfEPBl
⚠️ Erreur pour https://www.facebook.com/ibrahimamaigabf/posts/pfbid0rTqBUHiv3vZDBuaVzYPha6R8brJPzYnhkY1eDcSHUUi8PuApfQrHgcgCeTiQfEPBl: This API end-point can only be ac

KeyboardInterrupt: 

In [22]:
if all_comments:
    comments_df = pd.DataFrame(all_comments)
    comments_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
    print(f"\n✅ {len(all_comments)} commentaires enregistrés dans '{OUTPUT_CSV}'")
else:
    print("\n⚠️ Aucun commentaire collecté.")


✅ 965 commentaires enregistrés dans 'facebook_comments.csv'


In [23]:
comments = pd.read_csv("facebook_comments.csv")
comments.head(3)

,post_url,comment_id,comment_text,comment_author,comment_time,comment_reactions,reply_count
0,https://www.facebook.com/ibrahimamaigabf/posts...,Y29tbWVudDo0NjIwNTA5MDU5NzM3MTFfMTEyNzIyNDYyND...,Maiga il est temps de passer a l action car no...,NaN,NaN,NaN,NaN
1,https://www.facebook.com/ibrahimamaigabf/posts...,Y29tbWVudDo0NjIwNTA5MDU5NzM3MTFfODU3NTYwMzQxOD...,NaN,NaN,NaN,NaN,NaN
2,https://www.facebook.com/ibrahimamaigabf/posts...,Y29tbWVudDo0NjIwNTA5MDU5NzM3MTFfMTI4ODEzNTY3MT...,Affirmative 🙏,NaN,NaN,NaN,NaN


In [24]:
comments = comments[["post_url", "comment_text"]]
comments.columns = ["url", "comment_text"]
comments.head(3)

,url,comment_text
0,https://www.facebook.com/ibrahimamaigabf/posts...,Maiga il est temps de passer a l action car no...
1,https://www.facebook.com/ibrahimamaigabf/posts...,NaN
2,https://www.facebook.com/ibrahimamaigabf/posts...,Affirmative 🙏


In [ ]:
# merge data and comments